<h1><center> 2.3b Reduce solid waste per person</center></h1>

<p>Goal: general solid waste to region and county level</p>

In [1]:
import pandas as pd
#load population data
pop = pd.read_excel('MORPC_Region_County_Population_2000_2019_wide.xlsx')
pop.set_index(['COUNTY'],inplace = True)
pop.head()

,Id2,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
COUNTY,,,,,,,,,,,,,,,,,,,,,
DELAWARE,39041.0,111714,119098,127011,134899,141958,149353,155669,160698,164913,...,175098,178517,181124,185383,189451,193298,196923,200542,204826,208974
FAIRFIELD,39045.0,123414,125963,128860,131982,135055,137457,139503,141239,142299,...,146399,147141,147279,148783,150355,151202,152656,154557,155782,157155
FAYETTE,39047.0,28434,28170,28152,28133,28078,28212,28227,28272,28198,...,29017,28865,28765,28683,28647,28598,28658,28655,28666,28612
FRANKLIN,39049.0,1072307,1082421,1087369,1092518,1096256,1101224,1111018,1122357,1135643,...,1166261,1180741,1199071,1218985,1238527,1257552,1274580,1295706,1310300,1325083
HOCKING,39073.0,28237,28364,28511,28608,28687,28718,28699,28989,28968,...,29480,29466,29306,28653,28745,28498,28388,28457,28385,28362


<h3>Process the EPA Solid Waste data</h3>

In [2]:
df2006 = pd.read_excel('EPA Solid Waste 2006_2018.xlsx',str(2006))
df2006['year'] = 2006
df2006.head()

,SENDCNTY,COUNTY,Facility,TW_DH,TW_CoreID,GENERAL,ASBESTOS,RESIDUAL,INDUSTRIAL,CD_D,EXEMPT,RECYCLABLES,OTHER,year
0,21,DELAWARE,Richland County Transfer Station,NaN,0,1.86,0.0,0,0.0,0.00,0.0,0.0,0.0,2006
1,21,DELAWARE,Reynolds Avenue Transfer Station,NaN,0,31335.82,0.0,0,0.0,2413.33,0.0,0.0,0.0,2006
2,21,DELAWARE,Johnson Transfer & Recycling,NaN,0,7534.66,0.0,0,0.0,280.45,0.0,383.0,0.0,2006
3,21,DELAWARE,Rumpke Transfer Facility,NaN,0,22124.01,0.0,0,0.0,10.52,0.0,0.0,0.0,2006
4,21,DELAWARE,Ross Brothers Waste & Recycling Co.,NaN,0,1664.72,0.0,0,0.0,0.00,0.0,0.0,0.0,2006


In [4]:
generaldf = df2006.loc[:,['COUNTY','GENERAL','year']]
print(len(generaldf))
generaldf.dropna(subset=['COUNTY'],inplace = True)
generaldf['GENERAL'] = generaldf['GENERAL'].fillna(0)
print(len(generaldf))
general = generaldf[generaldf['COUNTY'].isin(['MARION','LOGAN', 'UNION', 'MADISON', 'FAYETTE', 'DELAWARE', 'MORROW', 'FRANKLIN','PICKAWAY', 'ROSS', 'KNOX', 'LICKING', 'FAIRFIELD', 'PERRY', 'HOCKING'])]
print(len(general))
df_sum = general.groupby(['COUNTY','year'])['GENERAL'].sum().reset_index()
df_sum.head()

213
213
208


,COUNTY,year,GENERAL
0,DELAWARE,2006,170617.29
1,FAIRFIELD,2006,149804.99
2,FAYETTE,2006,49911.93
3,FRANKLIN,2006,1718141.72
4,HOCKING,2006,27704.21


In [5]:
df_sum['GENERAL'] = df_sum.GENERAL.astype(float)
#recycle from tons to bound
df_sum['general_lbs'] = df_sum['GENERAL']*2000
df_sum.head()

,COUNTY,year,GENERAL,general_lbs
0,DELAWARE,2006,170617.29,3.412346e+08
1,FAIRFIELD,2006,149804.99,2.996100e+08
2,FAYETTE,2006,49911.93,9.982386e+07
3,FRANKLIN,2006,1718141.72,3.436283e+09
4,HOCKING,2006,27704.21,5.540842e+07


In [7]:
# region
region = df_sum.groupby(['year'])['general_lbs'].sum().reset_index()
region['pop'] = pop.loc['15-County Region',2006]
region['general_per_cap'] = region['general_lbs']/region['pop']
print(region)
final_region = region.loc[:,['year','general_per_cap']]
final_region

   year   general_lbs      pop  general_per_cap
0  2006  5.765883e+09  2074344      2779.617373


,year,general_per_cap
0,2006,2779.617373


In [8]:
#county
pop_county = pop.loc[:,['Id2',2006]]
pop_county.head()
county_merge = pd.merge(df_sum,pop_county,on='COUNTY')
county_merge['general_per_cap'] = county_merge['general_lbs']/county_merge[2006]
county_merge.head()

,COUNTY,year,GENERAL,general_lbs,Id2,2006,general_per_cap
0,DELAWARE,2006,170617.29,3.412346e+08,39041.0,155669,2192.052239
1,FAIRFIELD,2006,149804.99,2.996100e+08,39045.0,139503,2147.695605
2,FAYETTE,2006,49911.93,9.982386e+07,39047.0,28227,3536.467212
3,FRANKLIN,2006,1718141.72,3.436283e+09,39049.0,1111018,3092.914282
4,HOCKING,2006,27704.21,5.540842e+07,39073.0,28699,1930.674240


In [9]:
final_county = county_merge.loc[:,['Id2','year','general_per_cap']]
final_county['Id2'] =  final_county.Id2.astype(object)
final_county.head()

,Id2,year,general_per_cap
0,39041,2006,2192.052239
1,39045,2006,2147.695605
2,39047,2006,3536.467212
3,39049,2006,3092.914282
4,39073,2006,1930.674240


In [4]:
xls = pd.ExcelFile('EPA Solid Waste 2006_2018.xlsx')
pop = pd.read_excel('MORPC_Region_County_Population_2000_2019_wide.xlsx')
pop.set_index(['COUNTY'],inplace = True)
pop.rename(columns=lambda x: str(x), inplace=True)
#2006 to 2018
def readsheet(sheetname,pop):
    #process population
    df = pd.read_excel(xls, sheetname)
    df['year'] = sheetname
    df= df.loc[:,['COUNTY','GENERAL','year']]

    df.dropna(subset=['COUNTY'],inplace = True)
    df['GENERAL'] = df['GENERAL'].fillna(0)

    recycle = df[df['COUNTY'].isin(['MARION','LOGAN', 'UNION', 'MADISON', 'FAYETTE', 'DELAWARE', 'MORROW', 'FRANKLIN','PICKAWAY', 'ROSS', 'KNOX', 'LICKING', 'FAIRFIELD', 'PERRY', 'HOCKING'])]
    df_sum = recycle.groupby(['COUNTY','year'])['GENERAL'].sum().reset_index()
    df_sum['GENERAL'] = df_sum.GENERAL.astype(float)
    #recycle from tons to bound
    df_sum['general_lbs'] = df_sum['GENERAL']*2000
    #region
    region = df_sum.groupby(['year'])['general_lbs'].sum().reset_index()
    region['pop'] = pop.loc['15-County Region',sheetname]
    region['general_per_cap'] = region['general_lbs']/region['pop']
    final_region = region.loc[:,['year','general_per_cap']]
    #county
    pop_county = pop.loc[:,['Id2',sheetname]]
    county_merge = pd.merge(df_sum,pop_county,on='COUNTY')
    county_merge['general_per_cap'] = county_merge['general_lbs']/county_merge[sheetname]
    final_county = county_merge.loc[:,['Id2','year','general_per_cap']]
    final_county['Id2'] =  final_county.Id2.astype(object)
    
    return final_region, final_county

In [10]:
year = [ str(i) for i in range(2006,2019)]
df_region = pd.DataFrame()
df_county = pd.DataFrame()
for i in year:
    region,df = readsheet(i,pop)
    #df_county = pd.concat([df_county,df],ignore_index=True)
    df_county = df_county.append(df)
    df_region = df_region.append(region)


In [11]:
df_county

,Id2,year,general_per_cap
0,39041,2006,2192.052239
1,39045,2006,2147.695605
2,39047,2006,3536.467212
3,39049,2006,3092.914282
4,39073,2006,1930.674240
...,...,...,...
10,39117,2018,1942.481203
11,39127,2018,1507.916077
12,39129,2018,3621.084254
13,39141,2018,2789.221770


In [12]:
df_region

,year,general_per_cap
0,2006,2779.617373
0,2007,2636.592994
0,2008,2609.915036
0,2009,1983.382870
0,2010,1964.278536
0,2011,2279.104911
0,2012,2227.848409
0,2013,2201.549928
0,2014,2269.041643
0,2015,2314.768969
